# Introduction

Before applying any fancy algorithms, the next step is to simply explore whether the processed corpuses and DTMs make sense.

When working with numerical data, some EDA techniques we can use include finding the mean, median or mode as well as the distribution of a data set. 

For text, we are going to find some more **obvious** patterns with EDA before identifying the hidden patterns with machines learning (ML) techniques. Relatively obious things are:

1. **Most common words** - find these and create word clouds
2. **Size of vocabulary** - look number of unique words

## Outline for EDA
1. With the DTM from the previous stage, sort into columns according to ascending order in value (of occurence)
2. Aggregate (or filter) data - select columns with the largest values.
3. Visualise top words - word clouds? bar charts?
4. Insights - a written comment/ description of the key takeaways.
5. Probably also try using TF-IDF (Term frequency - inverse data frequency) for better understanding the "uniqueness" or the "value" of the the vocabulary in the text.